In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import pfba
from cobra.io import write_sbml_model,save_json_model,load_json_model,read_sbml_model
import re
import pandas as pd
import numpy as np
import xlrd
import re
import openpyxl
import sys
sys.path.append(r'./')
import os
from copy import copy, deepcopy
import numpy as np

In [2]:
model_cur_COMT_file='./cur/model/iML1515_iMl1515_cur_COMT.json'
model_cur_CcOAOMT_file='./cur/model/iML1515_iMl1515_cur_CcoAOMT.json'

In [3]:
model_origin_up='./cur/model/iML1515_new.json'
model_origin_down='./cur/model/iML1515_new.json'

In [4]:
model_new_up='./cur/model/iML1515_new_cur_up.json'
model_new_down='./cur/model/iML1515_new_cur_down.json'
model_new_up_CCoAOMT='./cur/model/iML1515_new_cur_up_CCoAOMT.json'
model_new_down_CCoAOMT='./cur/model/iML1515_new_cur_down_CCoAOMT.json'

# Refinement of model information

In [5]:
from model_correction_tools import *

In [6]:
model1=load_json_model(model_origin_up)
model2=load_json_model(model_origin_down)

add exogenous reactions

In [7]:
tyr__L_c=model1.metabolites.get_by_id('tyr__L_c')
nh4_c=model1.metabolites.get_by_id('nh4_c')
o2_c=model1.metabolites.get_by_id('o2_c')            
fad_c=model1.metabolites.get_by_id('fad_c')
h_c=model1.metabolites.get_by_id('h_c')
fadh2_c=model1.metabolites.get_by_id('fadh2_c')
h2o_c=model1.metabolites.get_by_id('h2o_c')

In [8]:
coumarate_c=Metabolite('coumarate_c',
                formula='C9H7O3',
                name='4-coumarate',
                compartment='c'
                )
coumarate_e=Metabolite('coumarate_e',
                formula='C9H7O3',
                name='4-coumarate',
                compartment='e'
                )
TAL=Reaction("TAL")
TAL.name='phenylalanine/tyrosine ammonia-lyase'
TAL.subsystem='unknow'
TAL.lower_bound=0
TAL.upper_bound=1000
TAL.add_metabolites({
    tyr__L_c:-1.0,
    coumarate_c:1.0,
    nh4_c:1.0
})
caffeate_c=Metabolite('caffeate_c',
                formula='C9H7O4',
                name='caffeic-acid',
                compartment='c'
                )
caffeate_e=Metabolite('caffeate_e',
                formula='C9H7O4',
                name='caffeic-acid',
                compartment='e'
                )      
C3H=Reaction("C3H")
C3H.name='4-coumarate 3-monooxygenaseInferred from experiment: hpaB'
C3H.subsystem='unknow'
C3H.lower_bound=0
C3H.upper_bound=1000
C3H.add_metabolites({
    o2_c:-1.0,
    fadh2_c:-1.0,
    coumarate_c:-1.0,
    fad_c:1.0,
    caffeate_c:1.0,
    h_c:1.0,
    h2o_c:1.0
})
amet_c=model1.metabolites.get_by_id('amet_c')
ahcys_c=model1.metabolites.get_by_id('ahcys_c')            
ferulate_c=Metabolite('ferulate_c',
                formula='C10H9O4',
                name='ferulic-acid',
                compartment='c'
                )
ferulate_e=Metabolite('ferulate_e',
                formula='C10H9O4',
                name='ferulic-acid',
                compartment='e'
                )     
COMT=Reaction("COMT")
COMT.name='caffeate 3-O-methyltransferaseInferred from experiment: COMT1'
COMT.subsystem='unknow'
COMT.lower_bound=0
COMT.upper_bound=1000
COMT.add_metabolites({
    caffeate_c:-1.0,
    amet_c:-1.0,
    ahcys_c:1.0,
    ferulate_c:1.0,
    h_c:1.0,
})

In [9]:
atp_c=model2.metabolites.get_by_id('atp_c')
coa_c=model2.metabolites.get_by_id('coa_c')
amp_c=model2.metabolites.get_by_id('amp_c')
ppi_c=model2.metabolites.get_by_id('ppi_c')
coumaroyl_CoA_c=Metabolite('coumaroyl_CoA_c',
                formula='C30H38N7O18P3S',
                name='coumaroyl-CoA',
                compartment='c'
                )
CL1=Reaction("4CL1")
CL1.name='4-coumarate:CoA ligase(4CL)_1'
CL1.subsystem='unknow'
CL1.lower_bound=0
CL1.upper_bound=1000
CL1.add_metabolites({
    coumarate_c:-1.0,
    atp_c:-1.0,
    coa_c:-1.0,
    amp_c:1.0,
    coumaroyl_CoA_c:1.0,
    ppi_c:1.0
})

In [10]:
caffeate_CoA_c=Metabolite('caffeate_CoA_c',
                formula='C30H38N7O19P3S',
                name='caffeoyl-CoA',
                compartment='c'
                )
CL2=Reaction("4CL2")
CL2.name='4-coumarate:CoA ligase(4CL)_2'
CL2.subsystem='unknow'
CL2.lower_bound=0
CL2.upper_bound=1000
CL2.add_metabolites({
    caffeate_c:-1.0,
    atp_c:-1.0,
    coa_c:-1.0,
    amp_c:1.0,
    caffeate_CoA_c:1.0,
    ppi_c:1.0
})

In [11]:
feruloyl_CoA_c=Metabolite('feruloyl_CoA_c',
                formula='C31H40N7O19P3S',
                name='feruloyl-CoA',
                compartment='c'
                )
CL3=Reaction("4CL3")
CL3.name='4-coumarate:CoA ligase(4CL)_3'
CL3.subsystem='unknow'
CL3.lower_bound=0
CL3.upper_bound=1000
CL3.add_metabolites({
    ferulate_c:-1.0,
    atp_c:-1.0,
    coa_c:-1.0,
    amp_c:1.0,
    feruloyl_CoA_c:1.0,
    ppi_c:1.0
})

In [12]:
malonyl_CoA_c=model2.metabolites.get_by_id('malcoa_c')
coumaroylacetyl_CoA_c=Metabolite('4_coumaroylacetyl_CoA_c',
                formula='C32H40N7O19P3S',
                name='4-coumaroylacetyl-CoA',
                compartment='c'
                )
coa_c=model2.metabolites.get_by_id('coa_c')
co2_c=model2.metabolites.get_by_id('co2_c')            
DCS=Reaction("DCS")
DCS.name='diketide-CoA synthaseInferred from experiment:DCS'
DCS.subsystem='unknow'
DCS.lower_bound=0
DCS.upper_bound=1000
DCS.add_metabolites({
    coumaroyl_CoA_c:-1.0,
    malonyl_CoA_c:-1.0,
    coumaroylacetyl_CoA_c:1.0,
    coa_c:1.0,
    co2_c:1.0,
})

In [13]:
feruloylacetyl_CoA_c=Metabolite('feruloylacetyl_CoA_c',
                formula='C33H42N7O20P3S',
                name='feruloylacetyl-CoA-CoA',
                compartment='c'
                )         
DCS2=Reaction("DCS2")
DCS2.name='diketide-CoA synthaseInferred from experiment:DCS2'
DCS2.subsystem='unknow'
DCS2.lower_bound=0
DCS2.upper_bound=1000
DCS2.add_metabolites({
    feruloyl_CoA_c:-1.0,
    malonyl_CoA_c:-1.0,
    feruloylacetyl_CoA_c:1.0,
    coa_c:1.0,
    co2_c:1.0,
})

In [14]:
h2o_c=model2.metabolites.get_by_id('h2o_c')
bisdemethoxycurcumin_c=Metabolite('bisdemethoxycurcumin_c',
                formula='C19H16O4',
                name='bisdemethoxycurcumin',
                compartment='c'
                )
bisdemethoxycurcumin_e=Metabolite('bisdemethoxycurcumin_e',
                formula='C19H16O4',
                name='bisdemethoxycurcumin',
                compartment='e'
                )            
CURS1=Reaction("CURS1")
CURS1.name='curcumin synthaseInferred from experiment: CURS 1'
CURS1.subsystem='unknow'
CURS1.lower_bound=0
CURS1.upper_bound=1000
CURS1.add_metabolites({
    coumaroyl_CoA_c:-1.0,
    coumaroylacetyl_CoA_c:-1.0,
    h2o_c:-1.0,
    bisdemethoxycurcumin_c:1.0,
    coa_c:2.0,
    co2_c:1.0,
})

In [15]:
demethoxycurcumin_c=Metabolite('demethoxycurcumin_c',
                formula='C20H18O5',
                name='demethoxycurcumin',
                compartment='c'
                )
demethoxycurcumin_e=Metabolite('demethoxycurcumin_e',
                formula='C20H18O5',
                name='demethoxycurcumin',
                compartment='e'
                )              
CURS2=Reaction("CURS2")
CURS2.name='curcumin synthaseInferred from experiment: CURS 2'
CURS2.subsystem='unknow'
CURS2.lower_bound=0
CURS2.upper_bound=1000
CURS2.add_metabolites({
    feruloyl_CoA_c:-1.0,
    coumaroylacetyl_CoA_c:-1.0,
    h2o_c:-1.0,
    demethoxycurcumin_c:1.0,
    coa_c:2.0,
    co2_c:1.0,
})

In [16]:
CURS3=Reaction("CURS3")
CURS3.name='curcumin synthaseInferred from experiment: CURS 3'
CURS3.subsystem='unknow'
CURS3.lower_bound=0
CURS3.upper_bound=1000
CURS3.add_metabolites({
    coumaroyl_CoA_c:-1.0,
    feruloylacetyl_CoA_c:-1.0,
    h2o_c:-1.0,
    demethoxycurcumin_c:1.0,
    coa_c:2.0,
    co2_c:1.0,
})

In [17]:
curcumin_c=Metabolite('curcumin_c',
                formula='C21H20O6',
                name='curcumin',
                compartment='c'
                )
curcumin_e=Metabolite('curcumin_e',
                formula='C21H20O6',
                name='curcumin',
                compartment='e'
                )        
CURS4=Reaction("CURS4")
CURS4.name='curcumin synthaseInferred from experiment: CURS 4'
CURS4.subsystem='unknow'
CURS4.lower_bound=0
CURS4.upper_bound=1000
CURS4.add_metabolites({
    feruloyl_CoA_c:-1.0,
    feruloylacetyl_CoA_c:-1.0,
    h2o_c:-1.0,
    curcumin_c:1.0,
    coa_c:2.0,
    co2_c:1.0,
})

Strategy I

In [18]:
exogenous_up=[TAL,C3H,COMT]
exogenous_down=[CL1,CL2,CL3,C3H,DCS,DCS2,CURS1,CURS2,CURS3,CURS4]

In [19]:
model1.add_reactions(exogenous_up)
model2.add_reactions(exogenous_down)

Strategy II

In [20]:
model3=deepcopy(model1)
model4=deepcopy(model2)

In [21]:
amet_c=model4.metabolites.get_by_id('amet_c')
ahcys_c=model4.metabolites.get_by_id('ahcys_c')            
CCoAOMT=Reaction("CCoAOMT")
CCoAOMT.name='caffeoyl-CoA-O-methyltransferaseInferred from experiment: CCoAOMT'
CCoAOMT.subsystem='unknow'
CCoAOMT.lower_bound=0
CCoAOMT.upper_bound=1000
CCoAOMT.add_metabolites({
    caffeate_CoA_c:-1.0,
    amet_c:-1.0,
    ahcys_c:1.0,
    feruloyl_CoA_c:1.0,
    h_c:1.0,
})

In [22]:
exogenous_up_new=[TAL,C3H]
exogenous_down_new=[CCoAOMT,CL1,CL2,CL3,C3H,DCS,DCS2,CURS1,CURS2,CURS3,CURS4]

In [23]:
model3.add_reactions(exogenous_up_new)
model4.add_reactions(exogenous_down_new)

Ignoring reaction 'TAL' since it already exists.
Ignoring reaction 'C3H' since it already exists.
Ignoring reaction '4CL1' since it already exists.
Ignoring reaction '4CL2' since it already exists.
Ignoring reaction '4CL3' since it already exists.
Ignoring reaction 'C3H' since it already exists.
Ignoring reaction 'DCS' since it already exists.
Ignoring reaction 'DCS2' since it already exists.
Ignoring reaction 'CURS1' since it already exists.
Ignoring reaction 'CURS2' since it already exists.
Ignoring reaction 'CURS3' since it already exists.
Ignoring reaction 'CURS4' since it already exists.


In [24]:
add_new_ex_special(model1,'coumarate','coumarate_c','coumarate_e')
add_new_ex_special(model1,'caffeate','caffeate_c','caffeate_e')
add_new_ex_special(model1,'ferulate','ferulate_c','ferulate_e')
add_new_ex_special(model2,'coumarate','coumarate_c','coumarate_e')
add_new_ex_special(model2,'caffeate','caffeate_c','caffeate_e')
add_new_ex_special(model2,'ferulate','ferulate_c','ferulate_e')
add_new_ex_special(model2,'bisdemethoxycurcumin','bisdemethoxycurcumin_c','bisdemethoxycurcumin_e')
add_new_ex_special(model2,'demethoxycurcumin','demethoxycurcumin_c','demethoxycurcumin_e')
add_new_ex_special(model2,'curcumin','curcumin_c','curcumin_e')

Name,iML1515
Memory address,0x0297cfe70048
Number of metabolites,1894
Number of reactions,2735
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [25]:
add_new_ex_special(model3,'coumarate','coumarate_c','coumarate_e')
add_new_ex_special(model3,'caffeate','caffeate_c','caffeate_e')
#add_new_ex_special(model3,'ferulate','ferulate_c','ferulate_e')
add_new_ex_special(model4,'coumarate','coumarate_c','coumarate_e')
add_new_ex_special(model4,'caffeate','caffeate_c','caffeate_e')
#add_new_ex_special(model4,'ferulate','ferulate_c','ferulate_e')
add_new_ex_special(model4,'bisdemethoxycurcumin','bisdemethoxycurcumin_c','bisdemethoxycurcumin_e')
add_new_ex_special(model4,'demethoxycurcumin','demethoxycurcumin_c','demethoxycurcumin_e')
add_new_ex_special(model4,'curcumin','curcumin_c','curcumin_e')

Name,iML1515
Memory address,0x0297d05e2358
Number of metabolites,1893
Number of reactions,2734
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


Test COMT/CCoAOMT Single-model

In [26]:
for i in ['coumarate_e','caffeate_e','ferulate_e']:
    model1.reactions.get_by_id('EX_'+i).bounds=(0,1000)
for j in ['EX_coumarate_e','EX_caffeate_e','EX_ferulate_e']:
    with model1 as model_1:
        model_1.objective=model_1.reactions.get_by_id(j)
        model_1.optimize()
        model_1.summary()


IN FLUXES        OUT FLUXES          OBJECTIVES
---------------  ------------------  --------------------
glc__D_e  10     h2o_e        37.6   EX_coumarate_e  5.61
o2_e       6.75  co2_e         9.55
                 coumarate_e   5.61
                 h_e           5.61
IN FLUXES       OUT FLUXES         OBJECTIVES
--------------  -----------------  -------------------
o2_e      11.7  h2o_e       38.5   EX_caffeate_e  5.37
glc__D_e  10    co2_e       11.7
                h_e         10.7
                caffeate_e   5.37
IN FLUXES     OUT FLUXES         OBJECTIVES
------------  -----------------  -------------------
o2_e      15  h2o_e       38.5   EX_ferulate_e  4.29
glc__D_e  10  co2_e       17.1
              h_e          8.58
              ferulate_e   4.29


In [27]:
for i in ['coumarate_e','caffeate_e','ferulate_e']:
    model2.reactions.get_by_id('EX_'+i).bounds=(-1000,0)
for j in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    with model2 as model_2:
        model_2.objective=model_2.reactions.get_by_id(j)
        model_2.optimize()
        model_2.summary()

IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  -------------------
coumarate_e  38  h2o_e            60  EX_bisdemeth...  19
o2_e         22  co2_e            41
h_e          19  bisdemethoxy...  19
glc__D_e     10
IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  -------------------
o2_e         22  h2o_e            60  EX_demethoxy...  19
coumarate_e  19  co2_e            41
ferulate_e   19  demethoxycur...  19
h_e          19
glc__D_e     10
IN FLUXES       OUT FLUXES      OBJECTIVES
--------------  --------------  -----------------
ferulate_e  38  h2o_e       60  EX_curcumin_e  19
o2_e        22  co2_e       41
h_e         19  curcumin_e  19
glc__D_e    10


In [28]:
for i in ['coumarate_e','caffeate_e']:
    model3.reactions.get_by_id('EX_'+i).bounds=(0,1000)
for j in ['EX_coumarate_e','EX_caffeate_e']:
    with model3 as model_1:
        model_1.objective=model_1.reactions.get_by_id(j)
        model_1.optimize()
        model_1.summary()

IN FLUXES        OUT FLUXES          OBJECTIVES
---------------  ------------------  --------------------
glc__D_e  10     h2o_e        37.6   EX_coumarate_e  5.61
o2_e       6.75  co2_e         9.55
                 h_e           5.61
                 coumarate_e   5.61
IN FLUXES       OUT FLUXES         OBJECTIVES
--------------  -----------------  -------------------
o2_e      11.7  h2o_e       38.5   EX_caffeate_e  5.37
glc__D_e  10    co2_e       11.7
                h_e         10.7
                caffeate_e   5.37


In [29]:
for i in ['coumarate_e','caffeate_e']:
    model4.reactions.get_by_id('EX_'+i).bounds=(-1000,0)
for j in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    with model4 as model_2:
        model_2.objective=model_2.reactions.get_by_id(j)
        model_2.optimize()
        model_2.summary()

IN FLUXES        OUT FLUXES           OBJECTIVES
---------------  -------------------  -------------------
coumarate_e  38  h2o_e            60  EX_bisdemeth...  19
o2_e         22  co2_e            41
h_e          19  bisdemethoxy...  19
glc__D_e     10
IN FLUXES          OUT FLUXES             OBJECTIVES
-----------------  ---------------------  ---------------------
o2_e         24.5  h2o_e            49.9  EX_demethoxy...  10.1
h_e          10.1  co2_e            39.7
caffeate_e   10.1  demethoxycur...  10.1
coumarate_e  10.1
glc__D_e     10
IN FLUXES          OUT FLUXES         OBJECTIVES
-----------------  -----------------  -------------------
o2_e        25.6   h2o_e       46.2   EX_curcumin_e  6.88
caffeate_e  13.8   co2_e       39.3
glc__D_e    10     curcumin_e   6.88
h_e          6.88


In [30]:
model1.optimize()
model1.summary()

IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ---------------------
o2_e      22.4       h2o_e  47.3   BIOMASS_Ec_i...  0.87
glc__D_e  10         co2_e  24.3
nh4_e      9.39      h_e     7.99
pi_e       0.839
so4_e      0.219
k_e        0.17
fe2_e      0.014
mg2_e      0.00755
ca2_e      0.00453
cl_e       0.00453
cu2_e      0.000617
mn2_e      0.000601
zn2_e      0.000297
ni2_e      0.000281


In [31]:
save_json_model(model1,model_new_up)
save_json_model(model2,model_new_down)
save_json_model(model3,model_new_up_CCoAOMT)
save_json_model(model4,model_new_down_CCoAOMT)

### Model Correction

SKMs Construction

In [32]:
from model_correction_tools import *

In [33]:
model_a=load_json_model(model_new_up)
model_b=load_json_model(model_new_down)
print("--------A  model----------")
print(len(model_a.reactions))
print(len(model_a.metabolites))
print(len(model_a.genes) )
print("--------B  model----------")
print(len(model_b.reactions))
print(len(model_b.metabolites))
print(len(model_b.genes) )
model=deepcopy(model_a)
for i in model.reactions:
    i.id='A_'+i.id
for i in model.metabolites:
    i.id='A_'+i.id
    i.name='A_'+i.name
model_t=deepcopy(model_b)
# for i in model_t.genes:
#     i.id=i.id+'_B'
for i in model_t.reactions:
    i.id='B_'+i.id
for i in model_t.metabolites:
    i.id='B_'+i.id
    i.name='B_'+i.name
for i in model_t.reactions:
    model.add_reaction(i)
e_meta_rn={}
for meta in model_a.metabolites:
    if meta.compartment=='e' :
        meta_a=model.metabolites.get_by_id('A_'+meta.id)
        e_meta_rn[meta.id+"_1"]=Reaction('A_'+meta.id+'_con_1')
        e_meta_rn[meta.id+"_1"].name = meta.id+' Connection 1'
        e_meta_rn[meta.id+"_1"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_1"].lower_bound = 0. # This is the default
        e_meta_rn[meta.id+"_1"].upper_bound = 1000. # This is the default
        e_meta_rn[meta.id+"_1"].add_metabolites({
            meta_a:-1.0,
            meta:1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_1"]])
for meta in model_b.metabolites:
    if meta.compartment=='e' :
        meta_b=model_t.metabolites.get_by_id('B_'+meta.id)
        e_meta_rn[meta.id+"_2"]=Reaction('B_'+meta.id+'_con_2')
        e_meta_rn[meta.id+"_2"].name = meta.id+' Connection 2'
        e_meta_rn[meta.id+"_2"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_2"].lower_bound = -1000. # This is the default
        e_meta_rn[meta.id+"_2"].upper_bound = 0. # This is the default
        e_meta_rn[meta.id+"_2"].add_metabolites({
            meta_b:1.0,
            meta:-1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_2"]])
origin_model1=cobra.io.load_json_model(model_new_up)
origin_model1.medium
for i in origin_model1.medium:
    model.add_reaction(origin_model1.reactions.get_by_id(i))
    ex_meta=i.split('EX_')[1]
    model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    #model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
    print('A_'+ex_meta+'_con_1')
origin_model2=cobra.io.load_json_model(model_new_down)
origin_model2.medium
for i in origin_model2.medium:
    model.add_reaction(origin_model2.reactions.get_by_id(i))
    ex_meta=i.split('EX_')[1]
    #model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
    print('B_'+ex_meta+'_con_2')
# model.reactions.get_by_id('A_'+meta_test+'_e'+'_con_1').bounds=(-1000,1000)
# model.reactions.get_by_id('B_'+meta_test+'_e'+'_con_2').bounds=(-1000,1000)
# for i in model.medium:
#     h=i.split('_')[0]
#     if h=='A' or h=='B':
#         model.reactions.get_by_id(i).bounds=(0,0)
model.medium
model.name='iML_iML_Cur'
model.id='iML_iML_Cur'
#model.medium


--------A  model----------
2722
1883
1516
--------B  model----------
2735
1894
1516
A_pi_e_con_1
A_h_e_con_1
A_fe3_e_con_1
A_mn2_e_con_1
A_co2_e_con_1
A_fe2_e_con_1
A_glc__D_e_con_1
A_zn2_e_con_1
A_mg2_e_con_1
A_ca2_e_con_1
A_ni2_e_con_1
A_cu2_e_con_1
A_cobalt2_e_con_1
A_sel_e_con_1
A_h2o_e_con_1
A_nh4_e_con_1
A_mobd_e_con_1
A_so4_e_con_1
A_k_e_con_1
A_na1_e_con_1
A_o2_e_con_1
A_cl_e_con_1
A_tungs_e_con_1
A_slnt_e_con_1


Ignoring reaction 'EX_pi_e' since it already exists.
Ignoring reaction 'EX_h_e' since it already exists.
Ignoring reaction 'EX_fe3_e' since it already exists.
Ignoring reaction 'EX_mn2_e' since it already exists.
Ignoring reaction 'EX_co2_e' since it already exists.
Ignoring reaction 'EX_fe2_e' since it already exists.
Ignoring reaction 'EX_glc__D_e' since it already exists.
Ignoring reaction 'EX_zn2_e' since it already exists.
Ignoring reaction 'EX_mg2_e' since it already exists.
Ignoring reaction 'EX_ca2_e' since it already exists.
Ignoring reaction 'EX_ni2_e' since it already exists.
Ignoring reaction 'EX_cu2_e' since it already exists.
Ignoring reaction 'EX_cobalt2_e' since it already exists.
Ignoring reaction 'EX_sel_e' since it already exists.
Ignoring reaction 'EX_h2o_e' since it already exists.
Ignoring reaction 'EX_nh4_e' since it already exists.
Ignoring reaction 'EX_mobd_e' since it already exists.
Ignoring reaction 'EX_so4_e' since it already exists.
Ignoring reaction 'EX_k

B_pi_e_con_2
B_h_e_con_2
B_fe3_e_con_2
B_mn2_e_con_2
B_co2_e_con_2
B_fe2_e_con_2
B_glc__D_e_con_2
B_zn2_e_con_2
B_mg2_e_con_2
B_ca2_e_con_2
B_ni2_e_con_2
B_cu2_e_con_2
B_cobalt2_e_con_2
B_sel_e_con_2
B_h2o_e_con_2
B_nh4_e_con_2
B_mobd_e_con_2
B_so4_e_con_2
B_k_e_con_2
B_na1_e_con_2
B_o2_e_con_2
B_cl_e_con_2
B_tungs_e_con_2
B_slnt_e_con_2
B_coumarate_e_con_2
B_caffeate_e_con_2
B_ferulate_e_con_2
B_bisdemethoxycurcumin_e_con_2
B_demethoxycurcumin_e_con_2
B_curcumin_e_con_2


Strategy I SKM

In [34]:
for i in model.medium:
    h=i.split('_')[0]
    if h=='A' or h=='B':
        model.reactions.get_by_id(i).bounds=(0,0)
model.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_coumarate_e': 1000,
 'EX_caffeate_e': 1000,
 'EX_ferulate_e': 1000,
 'EX_bisdemethoxycurcumin_e': 1000,
 'EX_demethoxycurcumin_e': 1000,
 'EX_curcumin_e': 1000}

In [35]:
for i in ['EX_coumarate_e','EX_caffeate_e','EX_ferulate_e','EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    model.reactions.get_by_id(i).bounds=(0,1000)

In [36]:
with model as model_test:
    for i in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
        model_test.reactions.get_by_id('A_BIOMASS_Ec_iML1515_core_75p37M').bounds=(0.1,1000)
        model_test.reactions.get_by_id('B_BIOMASS_Ec_iML1515_core_75p37M').bounds=(0.1,1000)
        model_test.objective=model_test.reactions.get_by_id(i)
        model_test.optimize()
        model_test.summary()
        pfba_solution = cobra.flux_analysis.pfba(model_test)
        for j in ['coumarate','caffeate','ferulate']:
            Find_ex_id='A_'+j+'_e_con_1'
            print(Find_ex_id+':'+str(pfba_solution.fluxes[Find_ex_id]))

IN FLUXES            OUT FLUXES              OBJECTIVES
-------------------  ----------------------  ---------------------
o2_e      13.3       h2o_e            42.6   EX_bisdemeth...  1.81
glc__D_e  10         co2_e            17.3
nh4_e      2.16      h_e               3.65
pi_e       0.193     bisdemethoxy...   1.81
so4_e      0.0504
k_e        0.039
fe2_e      0.00321
mg2_e      0.00174
cl_e       0.00104
ca2_e      0.00104
cu2_e      0.000142
mn2_e      0.000138
zn2_e      6.82e-05
ni2_e      6.46e-05
A_coumarate_e_con_1:3.6290222260803016
A_caffeate_e_con_1:0.0
A_ferulate_e_con_1:0.0
IN FLUXES            OUT FLUXES              OBJECTIVES
-------------------  ----------------------  --------------------
o2_e      16.2       h2o_e            42.7   EX_demethoxy...  1.6
glc__D_e  10         co2_e            19.8
nh4_e      2.16      h_e               5.04
pi_e       0.193     demethoxycur...   1.6
so4_e      0.0504
k_e        0.039
fe2_e      0.00321
mg2_e      0.00174
cl_e       0

In [37]:
with model as model_test:
    for i in ['A_BIOMASS_Ec_iML1515_core_75p37M','B_BIOMASS_Ec_iML1515_core_75p37M']:
        model_test.objective=model_test.reactions.get_by_id(i)
        #model_test.reactions.get_by_id('A_ATPM').bounds=(0,0)
        #model_test.objective=model_test.reactions.get_by_id(i)
        model_test.optimize()
        model_test.summary()

IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ----------------------
o2_e      25.5       h2o_e  51.5   A_BIOMASS_Ec...  0.844
glc__D_e  10         co2_e  25.4
nh4_e      9.11      fe3_e   7.75
fe2_e      7.77
pi_e       0.814
so4_e      0.212
k_e        0.165
mg2_e      0.00732
ca2_e      0.00439
cl_e       0.00439
cu2_e      0.000598
mn2_e      0.000583
zn2_e      0.000288
ni2_e      0.000272
IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ----------------------
o2_e      23.6       h2o_e  47.7   B_BIOMASS_Ec...  0.844
glc__D_e  10         co2_e  25.4
nh4_e      9.11      h_e     7.75
pi_e       0.814
so4_e      0.212
k_e        0.165
fe2_e      0.0135
mg2_e      0.00732
ca2_e      0.00439
cl_e       0.00439
cu2_e      0.000598
mn2_e      0.000583
zn2_e      0.000288
ni2_e      0.000272


In [38]:
save_json_model(model,model_cur_COMT_file)

Strategy II SKM

In [39]:
model_a=load_json_model(model_new_up_CCoAOMT)
model_b=load_json_model(model_new_down_CCoAOMT)
print("--------A  model----------")
print(len(model_a.reactions))
print(len(model_a.metabolites))
print(len(model_a.genes) )
print("--------B  model----------")
print(len(model_b.reactions))
print(len(model_b.metabolites))
print(len(model_b.genes) )
model=deepcopy(model_a)
for i in model.reactions:
    i.id='A_'+i.id
for i in model.metabolites:
    i.id='A_'+i.id
    i.name='A_'+i.name
model_t=deepcopy(model_b)
# for i in model_t.genes:
#     i.id=i.id+'_B'
for i in model_t.reactions:
    i.id='B_'+i.id
for i in model_t.metabolites:
    i.id='B_'+i.id
    i.name='B_'+i.name
for i in model_t.reactions:
    model.add_reaction(i)
e_meta_rn={}
for meta in model_a.metabolites:
    if meta.compartment=='e' :
        meta_a=model.metabolites.get_by_id('A_'+meta.id)
        e_meta_rn[meta.id+"_1"]=Reaction('A_'+meta.id+'_con_1')
        e_meta_rn[meta.id+"_1"].name = meta.id+' Connection 1'
        e_meta_rn[meta.id+"_1"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_1"].lower_bound = 0. # This is the default
        e_meta_rn[meta.id+"_1"].upper_bound = 1000. # This is the default
        e_meta_rn[meta.id+"_1"].add_metabolites({
            meta_a:-1.0,
            meta:1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_1"]])
for meta in model_b.metabolites:
    if meta.compartment=='e' :
        meta_b=model_t.metabolites.get_by_id('B_'+meta.id)
        e_meta_rn[meta.id+"_2"]=Reaction('B_'+meta.id+'_con_2')
        e_meta_rn[meta.id+"_2"].name = meta.id+' Connection 2'
        e_meta_rn[meta.id+"_2"].subsystem = 'unKnow'
        e_meta_rn[meta.id+"_2"].lower_bound = -1000. # This is the default
        e_meta_rn[meta.id+"_2"].upper_bound = 0. # This is the default
        e_meta_rn[meta.id+"_2"].add_metabolites({
            meta_b:1.0,
            meta:-1.0
            })
        model.add_reactions([e_meta_rn[meta.id+"_2"]])
origin_model1=cobra.io.load_json_model(model_new_up_CCoAOMT)
origin_model1.medium
for i in origin_model1.medium:
    model.add_reaction(origin_model1.reactions.get_by_id(i))
    ex_meta=i.split('EX_')[1]
    model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    #model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
    print('A_'+ex_meta+'_con_1')
origin_model2=cobra.io.load_json_model(model_new_down_CCoAOMT)
origin_model2.medium
for i in origin_model2.medium:
    model.add_reaction(origin_model2.reactions.get_by_id(i))
    ex_meta=i.split('EX_')[1]
    #model.reactions.get_by_id('A_'+ex_meta+'_con_1').bounds=(-1000,1000)
    model.reactions.get_by_id('B_'+ex_meta+'_con_2').bounds=(-1000,1000)
    print('B_'+ex_meta+'_con_2')
# model.reactions.get_by_id('A_'+meta_test+'_e'+'_con_1').bounds=(-1000,1000)
# model.reactions.get_by_id('B_'+meta_test+'_e'+'_con_2').bounds=(-1000,1000)
# for i in model.medium:
#     h=i.split('_')[0]
#     if h=='A' or h=='B':
#         model.reactions.get_by_id(i).bounds=(0,0)
model.medium
model.name='iML_iML_Cur'
model.id='iML_iML_Cur'
#model.medium

--------A  model----------
2720
1882
1516
--------B  model----------
2734
1893
1516
A_pi_e_con_1
A_h_e_con_1
A_fe3_e_con_1
A_mn2_e_con_1
A_co2_e_con_1
A_fe2_e_con_1
A_glc__D_e_con_1
A_zn2_e_con_1
A_mg2_e_con_1
A_ca2_e_con_1
A_ni2_e_con_1
A_cu2_e_con_1
A_cobalt2_e_con_1
A_sel_e_con_1
A_h2o_e_con_1
A_nh4_e_con_1
A_mobd_e_con_1
A_so4_e_con_1
A_k_e_con_1
A_na1_e_con_1
A_o2_e_con_1
A_cl_e_con_1
A_tungs_e_con_1
A_slnt_e_con_1


Ignoring reaction 'EX_pi_e' since it already exists.
Ignoring reaction 'EX_h_e' since it already exists.
Ignoring reaction 'EX_fe3_e' since it already exists.
Ignoring reaction 'EX_mn2_e' since it already exists.
Ignoring reaction 'EX_co2_e' since it already exists.
Ignoring reaction 'EX_fe2_e' since it already exists.
Ignoring reaction 'EX_glc__D_e' since it already exists.
Ignoring reaction 'EX_zn2_e' since it already exists.
Ignoring reaction 'EX_mg2_e' since it already exists.
Ignoring reaction 'EX_ca2_e' since it already exists.
Ignoring reaction 'EX_ni2_e' since it already exists.
Ignoring reaction 'EX_cu2_e' since it already exists.
Ignoring reaction 'EX_cobalt2_e' since it already exists.
Ignoring reaction 'EX_sel_e' since it already exists.
Ignoring reaction 'EX_h2o_e' since it already exists.
Ignoring reaction 'EX_nh4_e' since it already exists.
Ignoring reaction 'EX_mobd_e' since it already exists.
Ignoring reaction 'EX_so4_e' since it already exists.
Ignoring reaction 'EX_k

B_pi_e_con_2
B_h_e_con_2
B_fe3_e_con_2
B_mn2_e_con_2
B_co2_e_con_2
B_fe2_e_con_2
B_glc__D_e_con_2
B_zn2_e_con_2
B_mg2_e_con_2
B_ca2_e_con_2
B_ni2_e_con_2
B_cu2_e_con_2
B_cobalt2_e_con_2
B_sel_e_con_2
B_h2o_e_con_2
B_nh4_e_con_2
B_mobd_e_con_2
B_so4_e_con_2
B_k_e_con_2
B_na1_e_con_2
B_o2_e_con_2
B_cl_e_con_2
B_tungs_e_con_2
B_slnt_e_con_2
B_coumarate_e_con_2
B_caffeate_e_con_2
B_bisdemethoxycurcumin_e_con_2
B_demethoxycurcumin_e_con_2
B_curcumin_e_con_2


In [40]:
for i in model.medium:
    h=i.split('_')[0]
    if h=='A' or h=='B':
        model.reactions.get_by_id(i).bounds=(0,0)
model.medium

{'EX_pi_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0,
 'EX_coumarate_e': 1000,
 'EX_caffeate_e': 1000,
 'EX_bisdemethoxycurcumin_e': 1000,
 'EX_demethoxycurcumin_e': 1000,
 'EX_curcumin_e': 1000}

In [41]:
for i in ['EX_coumarate_e','EX_caffeate_e','EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    model.reactions.get_by_id(i).bounds=(0,1000)

In [42]:
with model as model_test:
    for i in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
        model_test.reactions.get_by_id('A_BIOMASS_Ec_iML1515_core_75p37M').bounds=(0.1,1000)
        model_test.reactions.get_by_id('B_BIOMASS_Ec_iML1515_core_75p37M').bounds=(0.1,1000)
        model_test.objective=model_test.reactions.get_by_id(i)
        model_test.optimize()
        model_test.summary()
        pfba_solution = cobra.flux_analysis.pfba(model_test)
        for j in ['coumarate','caffeate']:
            Find_ex_id='A_'+j+'_e_con_1'
            print(Find_ex_id+':'+str(pfba_solution.fluxes[Find_ex_id]))

IN FLUXES            OUT FLUXES              OBJECTIVES
-------------------  ----------------------  ---------------------
o2_e      13.3       h2o_e            42.6   EX_bisdemeth...  1.81
glc__D_e  10         co2_e            17.3
nh4_e      2.16      h_e               3.65
pi_e       0.193     bisdemethoxy...   1.81
so4_e      0.0504
k_e        0.039
fe2_e      0.00321
mg2_e      0.00174
cl_e       0.00104
ca2_e      0.00104
cu2_e      0.000142
mn2_e      0.000138
zn2_e      6.82e-05
ni2_e      6.46e-05
A_coumarate_e_con_1:3.629022226080304
A_caffeate_e_con_1:0.0
IN FLUXES            OUT FLUXES              OBJECTIVES
-------------------  ----------------------  --------------------
o2_e      16.1       h2o_e            42.7   EX_demethoxy...  1.6
glc__D_e  10         co2_e            19.8
nh4_e      2.16      h_e               5.04
pi_e       0.193     demethoxycur...   1.6
so4_e      0.0504
k_e        0.039
fe2_e      0.00321
mg2_e      0.00174
cl_e       0.00104
ca2_e      0.0010

In [43]:
with model as model_test:
    for i in ['A_BIOMASS_Ec_iML1515_core_75p37M','B_BIOMASS_Ec_iML1515_core_75p37M']:
        model_test.objective=model_test.reactions.get_by_id(i)
        #model_test.reactions.get_by_id('A_ATPM').bounds=(0,0)
        #model_test.objective=model_test.reactions.get_by_id(i)
        model_test.optimize()
        model_test.summary()

IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ----------------------
o2_e      25.5       h2o_e  51.5   A_BIOMASS_Ec...  0.844
glc__D_e  10         co2_e  25.4
nh4_e      9.11      fe3_e   7.75
fe2_e      7.77
pi_e       0.814
so4_e      0.212
k_e        0.165
mg2_e      0.00732
ca2_e      0.00439
cl_e       0.00439
cu2_e      0.000598
mn2_e      0.000583
zn2_e      0.000288
ni2_e      0.000272
IN FLUXES            OUT FLUXES    OBJECTIVES
-------------------  ------------  ----------------------
o2_e      23.6       h2o_e  47.7   B_BIOMASS_Ec...  0.844
glc__D_e  10         co2_e  25.4
nh4_e      9.11      h_e     7.75
pi_e       0.814
so4_e      0.212
k_e        0.165
fe2_e      0.0135
mg2_e      0.00732
ca2_e      0.00439
cl_e       0.00439
cu2_e      0.000598
mn2_e      0.000583
zn2_e      0.000288
ni2_e      0.000272


In [44]:
save_json_model(model,model_cur_CcOAOMT_file)

# Simulation

In [54]:
from CulECpyD import *
from model_ec_simulation_diff import *

Strategy I enzyme

In [46]:
model_file=model_cur_COMT_file
reaction_kcat_MW_file1='./cur/data/iML1515_reaction_kcat_MW.csv'
reaction_kcat_MW_file2='./cur/data/iML1515_reaction_kcat_MW2.csv'
reaction_kcat_MW_file3='./cur/data/iML1515_cur_COMT_reaction_kcat_MW.csv'
Concretemodel_Need_Data_COMT=Get_Concretemodel_Need_Data_json(model_file,reaction_kcat_MW_file1,'None')

Strategy II enzyme

In [47]:
model_file=model_cur_CcOAOMT_file
Concretemodel_Need_Data_CcOAOMT=Get_Concretemodel_Need_Data_json(model_file,reaction_kcat_MW_file1,'None')

In [50]:
Concretemodel_Need_Data_COMT['reaction_kcat_MW']=unpdate_kcat_mw_diff(Concretemodel_Need_Data_COMT['reaction_list'],reaction_kcat_MW_file1,reaction_kcat_MW_file2,reaction_kcat_MW_file3)
Concretemodel_Need_Data_CcOAOMT['reaction_kcat_MW']=unpdate_kcat_mw_diff(Concretemodel_Need_Data_CcOAOMT['reaction_list'],reaction_kcat_MW_file1,reaction_kcat_MW_file2,reaction_kcat_MW_file3)

Simulation result diff strategy

In [59]:
result_ec=ECM_FBA_diff(Concretemodel_Need_Data_COMT,substrates={'EX_glc__D_e_reverse':10},x=1,y=1,bio_a=0,bio_b=0,obj_name='A_BIOMASS_Ec_iML1515_core_75p37M',obj_target='maximize',special_constraint=True)
result_ec.obj()

0.6688780259185413

In [60]:
result_Cc_ec=ECM_FBA_diff(Concretemodel_Need_Data_CcOAOMT,substrates={'EX_glc__D_e_reverse':10},x=1,y=1,bio_a=0,bio_b=0,obj_name='A_BIOMASS_Ec_iML1515_core_75p37M',obj_target='maximize',special_constraint=True)
result_Cc_ec.obj()

0.6688780259185466

In [61]:
result_sto=FBA_diff(Concretemodel_Need_Data_COMT,substrates={'EX_glc__D_e_reverse':10},x=1,y=1,bio_a=0,bio_b=0,obj_name='A_BIOMASS_Ec_iML1515_core_75p37M',obj_target='maximize',special_constraint=True)
result_sto.obj()

0.8436355994032161

In [62]:
result_Cc_sto=FBA_diff(Concretemodel_Need_Data_CcOAOMT,substrates={'EX_glc__D_e_reverse':10},x=1,y=1,bio_a=0,bio_b=0,obj_name='A_BIOMASS_Ec_iML1515_core_75p37M',obj_target='maximize',special_constraint=True)
result_Cc_sto.obj()

0.8436355994032992

In [63]:
cur_result11=pd.DataFrame()
for j in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    result=ECM_FBA_diff(Concretemodel_Need_Data_COMT,substrates={'EX_glc__D_e_reverse':100},x=1,y=1,bio_a=0.1,bio_b=0.1,obj_name=j,obj_target='maximize',special_constraint=True)
    cur_result11.loc[j,'A_biomass']=(result.reaction['A_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result11.loc[j,'B_biomass']=(result.reaction['B_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result11.loc[j,'bisdemethoxycurcumin']=(result.reaction['EX_bisdemethoxycurcumin_e'].value)
    cur_result11.loc[j,'demethoxycurcumin']=(result.reaction['EX_demethoxycurcumin_e'].value)
    cur_result11.loc[j,'curcumin']=(result.reaction['EX_curcumin_e'].value)
cur_result11

,A_biomass,B_biomass,bisdemethoxycurcumin,demethoxycurcumin,curcumin
EX_bisdemethoxycurcumin_e,0.1,0.1,1.814511,0.000000,0.000000
EX_demethoxycurcumin_e,0.1,0.1,0.000000,0.082947,0.000000
EX_curcumin_e,0.1,0.1,0.000000,0.000000,0.041674


In [64]:
cur_result12=pd.DataFrame()
for j in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    result=ECM_FBA_diff(Concretemodel_Need_Data_COMT,substrates={'EX_glc__D_e_reverse':100},x=1,y=2,bio_a=0.1,bio_b=0.1,obj_name=j,obj_target='maximize',special_constraint=True)
    cur_result12.loc[j,'A_biomass']=(result.reaction['A_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result12.loc[j,'B_biomass']=(result.reaction['B_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result12.loc[j,'bisdemethoxycurcumin']=(result.reaction['EX_bisdemethoxycurcumin_e'].value)
    cur_result12.loc[j,'demethoxycurcumin']=(result.reaction['EX_demethoxycurcumin_e'].value)
    cur_result12.loc[j,'curcumin']=(result.reaction['EX_curcumin_e'].value)
cur_result12

,A_biomass,B_biomass,bisdemethoxycurcumin,demethoxycurcumin,curcumin
EX_bisdemethoxycurcumin_e,0.1,0.1,1.461038,0.000000,0.000000
EX_demethoxycurcumin_e,0.1,0.1,0.000000,0.082947,0.000000
EX_curcumin_e,0.1,0.1,0.000000,0.000000,0.041674


In [65]:
cur_result21=pd.DataFrame()
for j in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    result=ECM_FBA_diff(Concretemodel_Need_Data_COMT,substrates={'EX_glc__D_e_reverse':100},x=2,y=1,bio_a=0.1,bio_b=0.1,obj_name=j,obj_target='maximize',special_constraint=True)
    cur_result21.loc[j,'A_biomass']=(result.reaction['A_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result21.loc[j,'B_biomass']=(result.reaction['B_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result21.loc[j,'bisdemethoxycurcumin']=(result.reaction['EX_bisdemethoxycurcumin_e'].value)
    cur_result21.loc[j,'demethoxycurcumin']=(result.reaction['EX_demethoxycurcumin_e'].value)
    cur_result21.loc[j,'curcumin']=(result.reaction['EX_curcumin_e'].value)
cur_result21

,A_biomass,B_biomass,bisdemethoxycurcumin,demethoxycurcumin,curcumin
EX_bisdemethoxycurcumin_e,0.1,0.1,1.462539,0.000000,0.000000
EX_demethoxycurcumin_e,0.1,0.1,0.000000,0.165592,0.000000
EX_curcumin_e,0.1,0.1,0.000000,0.000000,0.083187


In [66]:
cur_result_Cc11=pd.DataFrame()
for j in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    result=ECM_FBA_diff(Concretemodel_Need_Data_CcOAOMT,substrates={'EX_glc__D_e_reverse':100},x=1,y=1,bio_a=0.1,bio_b=0.1,obj_name=j,obj_target='maximize',special_constraint=True)
    cur_result_Cc11.loc[j,'A_biomass']=(result.reaction['A_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result_Cc11.loc[j,'B_biomass']=(result.reaction['B_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result_Cc11.loc[j,'bisdemethoxycurcumin']=(result.reaction['EX_bisdemethoxycurcumin_e'].value)
    cur_result_Cc11.loc[j,'demethoxycurcumin']=(result.reaction['EX_demethoxycurcumin_e'].value)
    cur_result_Cc11.loc[j,'curcumin']=(result.reaction['EX_curcumin_e'].value)
cur_result_Cc11

,A_biomass,B_biomass,bisdemethoxycurcumin,demethoxycurcumin,curcumin
EX_bisdemethoxycurcumin_e,0.1,0.1,1.814511,0.000000,0.000000
EX_demethoxycurcumin_e,0.1,0.1,0.000000,0.053337,0.000000
EX_curcumin_e,0.1,0.1,0.000000,0.000000,0.026695


In [67]:
cur_result_Cc12=pd.DataFrame()
for j in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    result=ECM_FBA_diff(Concretemodel_Need_Data_CcOAOMT,substrates={'EX_glc__D_e_reverse':100},x=1,y=2,bio_a=0.1,bio_b=0.1,obj_name=j,obj_target='maximize',special_constraint=True)
    cur_result_Cc12.loc[j,'A_biomass']=(result.reaction['A_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result_Cc12.loc[j,'B_biomass']=(result.reaction['B_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result_Cc12.loc[j,'bisdemethoxycurcumin']=(result.reaction['EX_bisdemethoxycurcumin_e'].value)
    cur_result_Cc12.loc[j,'demethoxycurcumin']=(result.reaction['EX_demethoxycurcumin_e'].value)
    cur_result_Cc12.loc[j,'curcumin']=(result.reaction['EX_curcumin_e'].value)
cur_result_Cc12

,A_biomass,B_biomass,bisdemethoxycurcumin,demethoxycurcumin,curcumin
EX_bisdemethoxycurcumin_e,0.1,0.1,1.461038,0.000000,0.000000
EX_demethoxycurcumin_e,0.1,0.1,0.000000,0.106444,0.000000
EX_curcumin_e,0.1,0.1,0.000000,0.000000,0.053283


In [68]:
cur_result_Cc21=pd.DataFrame()
for j in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    result=ECM_FBA_diff(Concretemodel_Need_Data_CcOAOMT,substrates={'EX_glc__D_e_reverse':100},x=2,y=1,bio_a=0.1,bio_b=0.1,obj_name=j,obj_target='maximize',special_constraint=True)
    cur_result_Cc21.loc[j,'A_biomass']=(result.reaction['A_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result_Cc21.loc[j,'B_biomass']=(result.reaction['B_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result_Cc21.loc[j,'bisdemethoxycurcumin']=(result.reaction['EX_bisdemethoxycurcumin_e'].value)
    cur_result_Cc21.loc[j,'demethoxycurcumin']=(result.reaction['EX_demethoxycurcumin_e'].value)
    cur_result_Cc21.loc[j,'curcumin']=(result.reaction['EX_curcumin_e'].value)
cur_result_Cc21

,A_biomass,B_biomass,bisdemethoxycurcumin,demethoxycurcumin,curcumin
EX_bisdemethoxycurcumin_e,0.1,0.1,1.462539,0.000000,0.000000
EX_demethoxycurcumin_e,0.1,0.1,0.000000,0.053337,0.000000
EX_curcumin_e,0.1,0.1,0.000000,0.000000,0.026695


In [69]:
cur_result_fba=pd.DataFrame()#1:1
for j in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    result=FBA_diff(Concretemodel_Need_Data_COMT,substrates={'EX_glc__D_e_reverse':100},x=1,y=1,bio_a=0.1,bio_b=0.1,obj_name=j,obj_target='maximize',special_constraint=True)
    cur_result_fba.loc[j,'A_biomass']=(result.reaction['A_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result_fba.loc[j,'B_biomass']=(result.reaction['B_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result_fba.loc[j,'bisdemethoxycurcumin']=(result.reaction['EX_bisdemethoxycurcumin_e'].value)
    cur_result_fba.loc[j,'demethoxycurcumin']=(result.reaction['EX_demethoxycurcumin_e'].value)
    cur_result_fba.loc[j,'curcumin']=(result.reaction['EX_curcumin_e'].value)
cur_result_fba

,A_biomass,B_biomass,bisdemethoxycurcumin,demethoxycurcumin,curcumin
EX_bisdemethoxycurcumin_e,0.1,0.1,1.814512,0.000000,0.000000
EX_demethoxycurcumin_e,0.1,0.1,0.000000,1.600072,0.000000
EX_curcumin_e,0.1,0.1,0.000000,0.000000,1.430962


In [70]:
cur_result_fba_Cc=pd.DataFrame()#1:1
for j in ['EX_bisdemethoxycurcumin_e','EX_demethoxycurcumin_e','EX_curcumin_e']:
    result=FBA_diff(Concretemodel_Need_Data_CcOAOMT,substrates={'EX_glc__D_e_reverse':100},x=1,y=1,bio_a=0.1,bio_b=0.1,obj_name=j,obj_target='maximize',special_constraint=True)
    cur_result_fba_Cc.loc[j,'A_biomass']=(result.reaction['A_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result_fba_Cc.loc[j,'B_biomass']=(result.reaction['B_BIOMASS_Ec_iML1515_core_75p37M'].value)
    cur_result_fba_Cc.loc[j,'bisdemethoxycurcumin']=(result.reaction['EX_bisdemethoxycurcumin_e'].value)
    cur_result_fba_Cc.loc[j,'demethoxycurcumin']=(result.reaction['EX_demethoxycurcumin_e'].value)
    cur_result_fba_Cc.loc[j,'curcumin']=(result.reaction['EX_curcumin_e'].value)
cur_result_fba_Cc

,A_biomass,B_biomass,bisdemethoxycurcumin,demethoxycurcumin,curcumin
EX_bisdemethoxycurcumin_e,0.1,0.1,1.814512,0.000000,0.000000
EX_demethoxycurcumin_e,0.1,0.1,0.000000,1.602021,0.000000
EX_curcumin_e,0.1,0.1,0.000000,0.000000,1.432981
